# 단어의 표현(Word Representation)

기게는 문자를 그대로 인식할 수 없기 때문에 숫자로 반환

## 1. One-Hot

### 1.1, 직접 구현

"원숭이, 바나나, 사과" 로 원-핫 인코딩을 한다면

In [3]:
#인코딩 대상 단어들을 담은 리스트 
word_ls = ['원숭이','바나나','사과','사과'] 

In [4]:
from collections import defaultdict
import numpy as np

In [9]:
def one_hot_encode(word_ls):
    # 고유 단어와 인덱스를 매칭시켜주는 사전 생성 
    word2id_dic = defaultdict(lambda:len(word2id_dic))
    
    # {단어 : 인덱스} 사전 구축 for
    for word in word_ls:
        word2id_dic[word] 
        
    n_unique_words = len(word2id_dic) # 고유단어개수
    one_hot_vectors = np.zeros((len(word_ls), n_unique_words))
    
    for i, word in enumerate(word_ls):
        index = word2id_dic[word]# 해당 단어의 고유 인덱스 
        one_hot_vectors[i, index] = 1
    
    return one_hot_vectors
        
    

In [10]:
one_hot_vectors= one_hot_encode(word_ls) 

In [11]:
one_hot_vectors

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

"코끼리"라는 단어가 추가된다면?

In [15]:
word_ls = ['원숭이','바나나','사과','코끼리']

In [16]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## 1.3 sklearn 활용

In [17]:
# sklearn을 활용한 one-hot encoding
from numpy import array
from numpy import argmax # 확률로, 가장 큰 놈을 뽑아내는것
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [18]:
# 예제 데이터 배열
values = array(word_ls)
print(values)

['원숭이' '바나나' '사과' '코끼리']


In [19]:
# 문자열에 숫자를 붙임
label_enc = LabelEncoder()
int_enc = label_enc.fit_transform(values)
print(int_enc)

[2 0 1 3]


In [20]:
# binary encode
onehot_enc = OneHotEncoder(sparse=False) # Sparse True로 하면 index까지 표현
# onehot_enc = OneHotEncoder(sparse=True)
int_enc = int_enc.reshape(len(int_enc), 1) # n:1 matrix로 변환
print(int_enc)
onehot_enc = onehot_enc.fit_transform(int_enc)
print(onehot_enc)

[[2]
 [0]
 [1]
 [3]]
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [21]:
# one-hot encoding 의 첫번째 배열을 값을 역으로 산출
inverted = label_enc.inverse_transform([argmax(onehot_enc[0, :])]) # 첫번째 행
print(inverted)

['원숭이']


In [22]:
onehot_enc[0, :] # 0번째 행의 모든걸 가지고와~

array([0., 0., 1., 0.])

In [23]:
argmax(onehot_enc[0, :]) # 가장 큰 index를 가져와줘

2

## 2 밀집 벡터 (Dense Vector)

In [24]:
word_embedding_dic = {
'사과' : [1.0, 0.5],
'바나나' : [0.9, 1.2],
'원숭이' : [0.5, 1.5]
}

### 2-1 유사도 계산

### 2.1.1 유클리디언 거리(Euclidean distance)

두 벡터사이의 직선 거리. 피타고라스 정리를 생각하면 이해하기 쉬움

In [27]:
import numpy as np
def euclidean_dist(x,y): # np기반이기때문에 데이터를 다시 array로
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(np.sum(x-y)**2)

In [28]:
# 사과와 바나나의 유클리디안 유사도
euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나'])

0.6

### 2.1.2 코사인 유사도(Cosine Similarity)
- 두 벡터간의 유사도를 측정하는 방법 중 하나
- 두 벡터 사이의 코사인을 측정
- 0도 = 1, 90도 = 0, 180도 = -1 ==> 1에 가까울수록 유사도가 높음

In [29]:
def cosine_similarity(x, y):
# x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x, y) # 분자
    denominator = np.linalg.norm(x)*np.linalg.norm(y) # 분모
    return nominator/denominator

In [31]:
a =np.array([1, 2])
b = np.array([3, 4])
np.dot(a, b)

11

numpy의 linalg 서브 패키지의 norm 명령으로 벡터의 길이를 계산할 수 있다. 위에서 예로 든 2차원 벡터 𝑎=[1,2] 의 길
이는 √5≈2.236 이다.

In [32]:
np.linalg.norm(a)

2.23606797749979

In [33]:
# 사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))

0.8944271909999159
0.6


In [34]:
# 사과와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['원숭이']))

0.7071067811865475
0.5


In [36]:
# 바나나와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))

0.9486832980505138
0.09999999999999998


### 2.1.3 자카드 유사도(Jaccard index)

In [37]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

In [38]:
# 토큰화를 수행합니다.
token_s1 = s1.split()
token_s2 = s2.split()

union = set(token_s1).union(set(token_s2))
print(union)

intersection = set(token_s1).intersection(set(token_s2))
print(intersection)
      
print(len(intersection)/len(union)) # 2를 6로 나눔.

{'좋아합니다.', '대부분', '코주부', '바나나를', '싫어합니다.', '원숭이는'}
{'원숭이는', '바나나를'}
0.3333333333333333


In [39]:
## 

from collections import defaultdict
d_dict = defaultdict(int)

In [40]:
d_dict['a'] 
d_dict 

defaultdict(int, {'a': 0})

In [43]:
word2iddict = defaultdict(lambda : len(word2iddict))
# 호출될때마다 lambda 쓰겠다 이말이야!

In [44]:
word2iddict['a']

0

In [45]:
word2iddict['b']

1

In [46]:
word2iddict

defaultdict(<function __main__.<lambda>()>, {'a': 0, 'b': 1})